In [ ]:
#O(N)
import sys

def main():
    sys.setrecursionlimit(1000000000)
    with open('input.txt') as f:
        n = int(f.readline())
        if n == 1:
            print(0)
            return
        tree = [[] for _ in range(n + 1)]
        for _ in range(n - 1):
            u, v = map(int, f.readline().split())
            tree[u].append(v)
            tree[v].append(u)

    heights = {}
    diameters = {}

    def dfs_1(u, parent):
        max_heights = []
        heights[u] = 0
        diameters[u] = 0
        for v in tree[u]:
            if v != parent:
                dfs_1(v, u)
                heights[u] = max(heights[u], heights[v] + 1)
                max_heights.append(heights[v] + 1)
                diameters[u] = max(diameters[u], diameters[v])
        max_heights.sort(reverse=True)
        if len(max_heights) >= 2:
            diameters[u] = max(diameters[u], max_heights[0] + max_heights[1])
        elif max_heights:
            diameters[u] = max(diameters[u], max_heights[0])

    dfs_1(1, 0)

    ans = 0

    def dfs_2(u, parent, up_height, up_diameter):
        nonlocal ans
        child_heights = []
        child_diameters = []
        children = []
        for v in tree[u]:
            if v != parent:
                child_heights.append(heights[v] + 1)
                child_diameters.append(diameters[v])
                children.append(v)

        size = len(children)

        prefix_max_height = [0] * size
        suffix_max_height = [0] * size
        prefix_max_diameter = [0] * size
        suffix_max_diameter = [0] * size
        for i in range(size):
            h = child_heights[i]
            d = child_diameters[i]
            if i == 0:
                prefix_max_height[i] = h
                prefix_max_diameter[i] = d
            else:
                prefix_max_height[i] = max(prefix_max_height[i - 1], h)
                prefix_max_diameter[i] = max(prefix_max_diameter[i - 1], d)
        for i in range(size - 1, -1, -1):
            h = child_heights[i]
            d = child_diameters[i]
            if i == size - 1:
                suffix_max_height[i] = h
                suffix_max_diameter[i] = d
            else:
                suffix_max_height[i] = max(suffix_max_height[i + 1], h)
                suffix_max_diameter[i] = max(suffix_max_diameter[i + 1], d)

        for i, v in enumerate(children):
            up_height_v = up_height + 1
            if i > 0:
                up_height_v = max(up_height_v, prefix_max_height[i - 1] + 1)
            if i + 1 < size:
                up_height_v = max(up_height_v, suffix_max_height[i + 1] + 1)

            up_diameter_v = up_diameter
            if i > 0:
                up_diameter_v = max(up_diameter_v, prefix_max_diameter[i - 1])
            if i + 1 < size:
                up_diameter_v = max(up_diameter_v, suffix_max_diameter[i + 1])

            remaining_heights = []
            for j in range(size):
                if j != i:  
                    remaining_heights.append(child_heights[j])
            if up_height >= 0:
                remaining_heights.append(up_height)

            remaining_heights.sort(reverse=True)
            if len(remaining_heights) >= 2:
                max_sum_heights = remaining_heights[0] + remaining_heights[1]
            elif remaining_heights:
                max_sum_heights = remaining_heights[0]
            else:
                max_sum_heights = 0

            up_diameter_v = max(up_diameter_v, max_sum_heights)

            diameter1 = diameters[v]
            diameter2 = up_diameter_v

            ans = max(ans, diameter1 * diameter2)

            dfs_2(v, u, up_height_v, up_diameter_v)

    dfs_2(1, 0, 0, 0)
    print(ans)


if __name__ == "__main__":
    main()


In [9]:
#время O(N**2)
import sys
sys.setrecursionlimit(1000000000)
with open('input.txt') as file:
    n = int(file.readline())
    max_n = n + 10
    max_m = n * 2 + 10
    vertex = [0] * max_m  
    h = [-1] * max_n 
    visit = [False] * max_n   
    connection = [0] * max_m  
    del_edge = -1 
    idx = 0  
    res = 0   
    def dfs_1(u, fa):
        global res
        visit[u] = True
        dist = 0
        d1 = 0
        d2 = 0
        i = h[u]
        while i != -1:
            v = vertex[i]
            if v == fa or i == del_edge or (i ^ 1) == del_edge:
                i = connection[i]
                continue
            t = dfs_1(v, u) + 1
            dist = max(dist, t)
            if t >= d1:
                d2 = d1
                d1 = t
            elif t > d2:
                d2 = t
            i = connection[i]
        res = max(res, d1 + d2)
        return dist
    def add(u, v):
        global idx
        vertex[idx] = v
        connection[idx] = h[u]
        h[u] = idx
        idx += 1
    for _ in range(n-1):
        v1, v2 = map(int, file.readline().split())
        add(v1, v2)
        add(v2, v1)
    ans = 0
    for del_edge in range(0, idx, 2):
        temp = 1
        visit = [False] * (n + 1)
        for i in range(1, n + 1):
            if not visit[i]:
                res = 0
                dfs_1(i, -1)
                temp *= res
        ans = max(ans, temp)
    print(ans)

1
